Refernece : https://www.kaggle.com/kredy10/simple-lstm-for-text-classification/data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [3]:
df = pd.read_csv('spam.csv',delimiter=',',encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [4]:
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],axis=1,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
v1    5572 non-null object
v2    5572 non-null object
dtypes: object(2)
memory usage: 87.1+ KB


In [0]:
#process the labels
X = df.v2
Y = df.v1
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [0]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

In [0]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [0]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [9]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 150)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 150, 50)           50000     
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257       
__________

In [10]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Train on 3788 samples, validate on 948 samples
Epoch 1/10
3788/3788 [==============================] - 9s 2ms/step - loss: 0.3421 - acc: 0.8622 - val_loss: 0.1653 - val_acc: 0.9272
Epoch 2/10
3788/3788 [==============================] - 8s 2ms/step - loss: 0.0942 - acc: 0.9752 - val_loss: 0.0527 - val_acc: 0.9831
Epoch 3/10
3788/3788 [==============================] - 8s 2ms/step - loss: 0.0470 - acc: 0.9873 - val_loss: 0.0462 - val_acc: 0.9863
Epoch 4/10
3788/3788 [==============================] - 8s 2ms/step - loss: 0.0345 - acc: 0.9905 - val_loss: 0.0461 - val_acc: 0.9852


In [11]:
#evaluation on test data
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

accr = model.evaluate(test_sequences_matrix,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

836/836 [==============================] - 1s 783us/step
Test set
  Loss: 0.040
  Accuracy: 0.984


In [0]:
# 1Dcon with lstm
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding

In [0]:
def create_conv_model():
    model_conv = Sequential()
    model_conv.add(Embedding(max_words,50,input_length=max_len))
    model_conv.add(Dropout(0.2))
    model_conv.add(Conv1D(64, 5, activation='relu'))
    model_conv.add(MaxPooling1D(pool_size=4))
    model_conv.add(LSTM(100))
    model_conv.add(Dense(1, activation='sigmoid'))
    model_conv.compile(loss='binary_crossentropy', optimizer='adam',    metrics=['accuracy'])
    return model_conv

In [19]:
#5 epochs
model_conv = create_conv_model()
model_conv.fit(sequences_matrix,Y_train,batch_size=128,epochs=5,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Train on 3788 samples, validate on 948 samples
Epoch 1/5
3788/3788 [==============================] - 7s 2ms/step - loss: 0.4284 - acc: 0.8677 - val_loss: 0.3197 - val_acc: 0.8681
Epoch 2/5
3788/3788 [==============================] - 6s 1ms/step - loss: 0.2411 - acc: 0.8978 - val_loss: 0.1390 - val_acc: 0.9715
Epoch 3/5
3788/3788 [==============================] - 6s 1ms/step - loss: 0.0890 - acc: 0.9744 - val_loss: 0.0781 - val_acc: 0.9778
Epoch 4/5
3788/3788 [==============================] - 6s 1ms/step - loss: 0.0513 - acc: 0.9855 - val_loss: 0.0727 - val_acc: 0.9831
Epoch 5/5
3788/3788 [==============================] - 6s 1ms/step - loss: 0.0360 - acc: 0.9905 - val_loss: 0.0717 - val_acc: 0.9842


In [20]:
#10 epochs
model_conv = create_conv_model()
model_conv.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Train on 3788 samples, validate on 948 samples
Epoch 1/10
3788/3788 [==============================] - 7s 2ms/step - loss: 0.4603 - acc: 0.8453 - val_loss: 0.3473 - val_acc: 0.8681
Epoch 2/10
3788/3788 [==============================] - 6s 1ms/step - loss: 0.2792 - acc: 0.8796 - val_loss: 0.1712 - val_acc: 0.9546
Epoch 3/10
3788/3788 [==============================] - 6s 1ms/step - loss: 0.1047 - acc: 0.9694 - val_loss: 0.0758 - val_acc: 0.9757
Epoch 4/10
3788/3788 [==============================] - 6s 1ms/step - loss: 0.0553 - acc: 0.9834 - val_loss: 0.0694 - val_acc: 0.9831
Epoch 5/10
3788/3788 [==============================] - 6s 1ms/step - loss: 0.0398 - acc: 0.9886 - val_loss: 0.0620 - val_acc: 0.9852
Epoch 6/10
3788/3788 [==============================] - 6s 1ms/step - loss: 0.0305 - acc: 0.9918 - val_loss: 0.0651 - val_acc: 0.9842


In [21]:
#evaluation on test data
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

accr = model_conv.evaluate(test_sequences_matrix,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

836/836 [==============================] - 0s 512us/step
Test set
  Loss: 0.060
  Accuracy: 0.981
